In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import seaborn as sn
import matplotlib.pyplot as plt
from collections import Counter
from imblearn.over_sampling import SMOTE
from scipy import stats
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import AdaBoostRegressor,RandomForestRegressor,GradientBoostingRegressor,BaggingRegressor
from sklearn.neighbors import KNeighborsRegressor
from reg_resampler import resampler
from xgboost import XGBRegressor
from catboost import CatBoostRegressor
from ngboost import NGBRegressor
from lightgbm import LGBMRegressor

Using TensorFlow backend.


In [8]:
df_train = pd.read_csv("Train.csv")
train_view = df_train.copy()
df_test = pd.read_csv("Test.csv")
display(df_train.shape)
df_train['is_train'] = 1
df_test['is_train']  = 0
df_train = df_train.append(df_test).reset_index(drop = True)
#-------------------------------
# ohe = OneHotEncoder()
# CONSOLE = ohe.fit_transform(df_train["RATING"].values.reshape(-1,1))
# df_train = df_train.join(pd.DataFrame(CONSOLE.toarray(),columns = ohe.get_feature_names()).astype(int))


#------------Remove outlier-------------------
df_train=df_train.drop(1036,axis=0)
#------------------------------features---------------------------------------
df_train["YEAR"] = df_train["YEAR"].max() - df_train["YEAR"]
#------------------------------------------------------------
df_train["AVG_POINTS"] = (df_train["CRITICS_POINTS"]*df_train["USER_POINTS"])*100
#------------------------------------------------------------
feature = df_train.groupby(['CONSOLE','PUBLISHER']).agg({'CRITICS_POINTS': ['max','min','count','sum'],
                                                        'USER_POINTS': ['max','min','count','sum']})
feature.columns = ['_'.join(c) for c in feature.columns]
df_train = pd.merge(df_train, feature, on = ['CONSOLE','PUBLISHER'], how='left')
#------------------------------------------------------------
feature = df_train.groupby(['CATEGORY','PUBLISHER']).agg({'CRITICS_POINTS': ['max','min','count','sum'],
                                                        'USER_POINTS': ['max','min','count','sum']})
feature.columns = ['_CATPUB_'.join(c) for c in feature.columns]
df_train = pd.merge(df_train, feature, on = ['CATEGORY','PUBLISHER'], how='left')
#------------------------------------------------------------
feature = df_train.groupby(['PUBLISHER']).agg({'CRITICS_POINTS': ['max','min','count','sum'],
                                                        'USER_POINTS': ['max','min','count','sum']})
feature.columns = ['_PUB_'.join(c) for c in feature.columns]
df_train = pd.merge(df_train, feature, on = ['PUBLISHER'], how='left')
#------------------------------------------------------------
feature = df_train.groupby(['CATEGORY','RATING']).agg({'CRITICS_POINTS': ['max','min','count','sum'],
                                                        'USER_POINTS': ['max','min','count','sum']})
feature.columns = ['_CATRAT_'.join(c) for c in feature.columns]
df_train = pd.merge(df_train, feature, on = ['CATEGORY','RATING'], how='left')
#------------------------------------------------------------
#------------------------------------------------------------
# feature = df_train.groupby(['CATEGORY','YEAR']).agg({'CRITICS_POINTS': ['max','min'],
#                                                         'USER_POINTS': ['max','min','sum']})
# feature.columns = ['_CATYEAR_'.join(c) for c in feature.columns]
# df_train = pd.merge(df_train, feature, on = ['CATEGORY','YEAR'], how='left')

#---------------------------------------------------------------------------------
le = LabelEncoder()
df_train["CONSOLE"] = le.fit_transform(df_train["CONSOLE"])
df_train["CATEGORY"] = le.fit_transform(df_train["CATEGORY"])
df_train["PUBLISHER"] = le.fit_transform(df_train["PUBLISHER"])
df_train["RATING"] = le.fit_transform(df_train["RATING"])
#---------------------------------------------------------------------------------

df_test = df_train[df_train['is_train'] == 0]
df_test.drop("is_train",axis=1,inplace=True)
# df_test.drop("RATING",axis=1,inplace=True)

df_train = df_train[df_train['is_train'] == 1]
df_train.drop("is_train",axis=1,inplace=True)
# df_train.drop("RATING",axis=1,inplace=True)


(3506, 9)

In [9]:
# display(df_train.drop(['ID'],axis=1))
# final_test
df_train

,ID,CONSOLE,YEAR,CATEGORY,PUBLISHER,RATING,CRITICS_POINTS,USER_POINTS,SalesInMillions,AVG_POINTS,...,USER_POINTS_PUB_count,USER_POINTS_PUB_sum,CRITICS_POINTS_CATRAT_max,CRITICS_POINTS_CATRAT_min,CRITICS_POINTS_CATRAT_count,CRITICS_POINTS_CATRAT_sum,USER_POINTS_CATRAT_max,USER_POINTS_CATRAT_min,USER_POINTS_CATRAT_count,USER_POINTS_CATRAT_sum
0,2860,2,11,7,140,1,2.833333,0.303704,1.779257,86.049383,...,250,50.167558,13.75,1.086957,55,230.049453,2.200,0.014426,55,23.029137
1,731,12,7,9,110,2,13.200000,1.640000,0.215050,2164.800000,...,162,81.942478,13.20,0.853333,19,93.724409,1.760,0.000481,19,13.881816
2,495,5,0,8,8,4,4.562500,0.006410,0.534402,2.924679,...,397,170.329758,20.75,0.644737,381,894.628219,2.125,0.000390,381,64.139034
3,2641,7,17,10,56,1,4.181818,0.326923,1.383964,136.713287,...,725,269.745314,19.00,1.068493,545,2498.383806,2.325,0.004767,545,303.638562
4,811,8,6,0,8,4,2.259259,0.032579,0.082671,7.360483,...,397,170.329758,18.25,0.568966,431,1092.174662,2.200,0.000990,431,68.655386
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3500,1323,2,12,7,186,2,1.702128,0.065421,3.584264,11.135415,...,98,14.518229,15.40,1.137931,71,216.503104,1.250,0.011568,71,23.907360
3501,2862,0,1,0,140,2,3.625000,0.090000,2.701472,32.625000,...,250,50.167558,21.50,0.800000,232,1269.630228,2.325,0.004006,232,128.198871
3502,1980,8,7,0,223,2,2.814815,0.172917,1.631668,48.672840,...,111,33.333124,21.50,0.800000,232,1269.630228,2.325,0.004006,232,128.198871
3503,3065,0,3,9,118,2,2.571429,0.464286,2.899156,119.387755,...,3,0.813210,13.20,0.853333,19,93.724409,1.760,0.000481,19,13.881816


In [10]:
# train_view[(train_view["CATEGORY"]=="role-playing") & (train_view["YEAR"]==2012)]

In [21]:
#outlier
df_train[df_train["SalesInMillions"]>40]

,ID,CONSOLE,YEAR,CATEGORY,PUBLISHER,RATING,CRITICS_POINTS,USER_POINTS,SalesInMillions,AVG_POINTS,...,USER_POINTS_PUB_count,USER_POINTS_PUB_sum,CRITICS_POINTS_CATRAT_max,CRITICS_POINTS_CATRAT_min,CRITICS_POINTS_CATRAT_count,CRITICS_POINTS_CATRAT_sum,USER_POINTS_CATRAT_max,USER_POINTS_CATRAT_min,USER_POINTS_CATRAT_count,USER_POINTS_CATRAT_sum


In [22]:
features = [col for col in df_train.columns if col not in ("SalesInMillions","ID")]
final_test= df_test[features]
trainx = df_train.drop(['SalesInMillions','ID'],axis=1)
trainy = df_train['SalesInMillions']
# trainx,testx,trainy,testy = train_test_split(df_train.drop('SalesInMillions',axis=1),
#                                              df_train['SalesInMillions'],test_size = 0.2,
#                                              shuffle = True,random_state=27)
catfeatures =["CONSOLE","CATEGORY","PUBLISHER","RATING"]

In [25]:
final_test

,CONSOLE,YEAR,CATEGORY,PUBLISHER,RATING,CRITICS_POINTS,USER_POINTS,AVG_POINTS,CRITICS_POINTS_max,CRITICS_POINTS_min,...,USER_POINTS_PUB_count,USER_POINTS_PUB_sum,CRITICS_POINTS_CATRAT_max,CRITICS_POINTS_CATRAT_min,CRITICS_POINTS_CATRAT_count,CRITICS_POINTS_CATRAT_sum,USER_POINTS_CATRAT_max,USER_POINTS_CATRAT_min,USER_POINTS_CATRAT_count,USER_POINTS_CATRAT_sum
3505,7,11,0,202,6,3.928571,0.482353,189.495798,9.571429,1.600000,...,90,45.138836,20.75,0.711864,414,1398.921478,1.875000,0.000874,414,157.990157
3506,10,12,11,21,2,5.538462,0.071795,39.763314,5.538462,1.818182,...,127,63.595236,16.50,1.047619,47,182.581462,1.216667,0.003090,47,14.891719
3507,7,15,8,56,6,3.034483,0.062044,18.827083,16.750000,1.339286,...,725,269.745314,15.00,0.800000,201,615.445310,2.000000,0.001091,201,74.675552
3508,14,13,0,56,1,2.913043,0.880000,256.347826,8.000000,1.160714,...,725,269.745314,20.50,0.964912,137,748.660777,2.250000,0.008318,137,76.063492
3509,0,8,6,212,2,1.162162,0.183333,21.306306,11.500000,1.162162,...,357,126.673153,18.25,1.011765,59,234.033860,1.875000,0.006033,59,21.357628
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5003,7,15,10,56,6,1.809524,0.192308,34.798535,16.750000,1.339286,...,725,269.745314,17.75,1.200000,75,344.788858,2.250000,0.015909,75,37.089932
5004,2,12,4,198,1,8.800000,0.625000,550.000000,17.750000,1.094118,...,212,53.399040,17.75,0.901408,180,813.187587,2.325000,0.001278,180,92.820996
5005,16,3,0,101,1,5.500000,0.333333,183.333333,5.500000,5.500000,...,3,0.954735,20.50,0.964912,137,748.660777,2.250000,0.008318,137,76.063492
5006,6,17,6,56,1,8.625000,0.690000,595.125000,13.500000,3.187500,...,725,269.745314,20.00,0.835294,247,1125.415696,2.200000,0.003863,247,128.443078


## CatBoost

In [28]:
kfold, scores = KFold(n_splits=5, shuffle=True, random_state=27), []
Actual_Preds=[]
i=1
for train_, test_ in kfold.split(trainx):
        x_train, x_test = trainx.iloc[train_], trainx.iloc[test_]
        y_train, y_test = trainy.iloc[train_], trainy.iloc[test_]
        
        model = CatBoostRegressor(verbose=0,cat_features = catfeatures,
                                  random_state=27,l2_leaf_reg=0.02)
        model.fit(x_train,y_train,eval_set=(x_test,y_test))

        preds = model.predict(x_test)
        score = mean_squared_error(y_test, preds)**(1/2)
        scores.append(score)
        realpreds = model.predict(final_test)
        Actual_Preds.append(realpreds)
        print(score)
print("AVERAGE SCORE: ", sum(scores)/len(scores))

1.4372665552240071
1.8624128882088202
1.7906848192386466
1.591021142480063
1.544428718868214
AVERAGE SCORE:  1.64516282480395


### Using reg_resampler for Sampling the data!
#### Developed by : Me and AtifHassan  
[link to the package](https://pypi.org/project/reg-resampler/)

In [26]:
rs = resampler()
send_resample = df_train.drop(['ID'],axis=1)
Y_classes = rs.fit(send_resample,"SalesInMillions",bins=10)

INFO: Class 4 has been merged into Class 3 due to low number of samples
INFO: Class 5 has been merged into Class 3 due to low number of samples
INFO: Class 6 has been merged into Class 3 due to low number of samples
INFO: Class 7 has been merged into Class 3 due to low number of samples

Class Distribution:
-------------------
0: 3110
1: 320
2: 48
3: 27



In [27]:
kfold, scores = KFold(n_splits=5, shuffle=True, random_state=27), []
Actual_Preds=[]
i=1
for train_, test_ in kfold.split(send_resample):
        x_train, y_train = send_resample.iloc[train_], Y_classes[train_]
        x_test, y_test = send_resample.iloc[test_], trainy.iloc[test_]
    
        # Remove the target variable from x_test
        x_test = x_test.drop("SalesInMillions", axis=1)
        
        xp = Counter(y_train)
        # Your favourite oversampler
        smote = SMOTE(random_state=27, sampling_strategy={3:int(xp[3]*4),
                                                          2: int(xp[2]*1),
                                                          0: int(xp[0]*1)})
        # just to see default
#         smote = SMOTE(random_state=27, sampling_strategy={3:int(xp[3]*1), 2: int(xp[2]*1)})

        # Generate the over-sampled data
        x_train, y_train = rs.resample(smote, x_train, y_train)
        

        model = CatBoostRegressor(verbose=0,cat_features = catfeatures,
                                  random_state=27,l2_leaf_reg=0.02)
        model.fit(x_train,y_train,eval_set=(x_test,y_test))

        preds = model.predict(x_test)
        score = mean_squared_error(y_test, preds)**(1/2)
        scores.append(score)
        realpreds = model.predict(final_test)
        Actual_Preds.append(realpreds)
        print(score)
print("AVERAGE SCORE: ", sum(scores)/len(scores))

1.4724018819585964
1.9042449236873176
1.6951420478013703
1.6431096102024334
1.4809465031052835
AVERAGE SCORE:  1.639168993351


In [140]:
preds = np.mean(Actual_Preds,0)
op = pd.DataFrame(preds,columns=["SalesInMillions"])

## LGBM

In [30]:
df_train = pd.read_csv("Train.csv")
train_view = df_train.copy()
df_test = pd.read_csv("Test.csv")
display(df_train.shape)
df_train['is_train'] = 1
df_test['is_train']  = 0
df_train = df_train.append(df_test).reset_index(drop = True)
#-------------------------------
# ohe = OneHotEncoder()
# CONSOLE = ohe.fit_transform(df_train["RATING"].values.reshape(-1,1))
# df_train = df_train.join(pd.DataFrame(CONSOLE.toarray(),columns = ohe.get_feature_names()).astype(int))


#------------Remove outlier-------------------
df_train=df_train.drop(1036,axis=0)
#------------------------------features---------------------------------------
df_train["YEAR"] = df_train["YEAR"].max() - df_train["YEAR"]
#------------------------------------------------------------
df_train["AVG_POINTS"] = (df_train["CRITICS_POINTS"]*df_train["USER_POINTS"])*100
#------------------------------------------------------------
feature = df_train.groupby(['CONSOLE','PUBLISHER']).agg({'CRITICS_POINTS': ['max','min','count','sum'],
                                                        'USER_POINTS': ['max','min','count','sum']})
feature.columns = ['_'.join(c) for c in feature.columns]
df_train = pd.merge(df_train, feature, on = ['CONSOLE','PUBLISHER'], how='left')
#------------------------------------------------------------
feature = df_train.groupby(['CATEGORY','PUBLISHER']).agg({'CRITICS_POINTS': ['max','min','count','sum'],
                                                        'USER_POINTS': ['max','min','count','sum']})
feature.columns = ['_CATPUB_'.join(c) for c in feature.columns]
df_train = pd.merge(df_train, feature, on = ['CATEGORY','PUBLISHER'], how='left')
#------------------------------------------------------------
feature = df_train.groupby(['PUBLISHER']).agg({'CRITICS_POINTS': ['max','min','count','sum'],
                                                        'USER_POINTS': ['max','min','count','sum']})
feature.columns = ['_PUB_'.join(c) for c in feature.columns]
df_train = pd.merge(df_train, feature, on = ['PUBLISHER'], how='left')
#------------------------------------------------------------
feature = df_train.groupby(['CATEGORY','RATING']).agg({'CRITICS_POINTS': ['max','min','count','sum'],
                                                        'USER_POINTS': ['max','min','count','sum']})
feature.columns = ['_CATRAT_'.join(c) for c in feature.columns]
df_train = pd.merge(df_train, feature, on = ['CATEGORY','RATING'], how='left')
#------------------------------------------------------------
feature = df_train.groupby(['ID']).agg({'CRITICS_POINTS': ["count"]})
feature.columns = ['_ID_'.join(c) for c in feature.columns]
df_train = pd.merge(df_train, feature, on = ['ID'], how='left')

#------------------------------------------------------------

#---------------------------------------------------------------------------------
le = LabelEncoder()
df_train["CONSOLE"] = le.fit_transform(df_train["CONSOLE"])
df_train["CATEGORY"] = le.fit_transform(df_train["CATEGORY"])
df_train["PUBLISHER"] = le.fit_transform(df_train["PUBLISHER"])
df_train["RATING"] = le.fit_transform(df_train["RATING"])
#---------------------------------------------------------------------------------

df_test = df_train[df_train['is_train'] == 0]
df_test.drop("is_train",axis=1,inplace=True)

df_train = df_train[df_train['is_train'] == 1]
df_train.drop("is_train",axis=1,inplace=True)


(3506, 9)

In [31]:
#Dropping Category and Publisher gave good score for LGBM
features = [col for col in df_train.columns if col not in ("SalesInMillions","CATEGORY","PUBLISHER")]
final_test= df_test[features]
trainx = df_train[features]
trainy = df_train['SalesInMillions']
catfeatures =["CONSOLE","CATEGORY","PUBLISHER","RATING"]

### KFOLD

In [34]:
kfold, scores = KFold(n_splits=5, shuffle=True, random_state=27), []
Actual_Preds=[]
i=1
for train_, test_ in kfold.split(trainx):
        x_train, x_test = trainx.iloc[train_], trainx.iloc[test_]
        y_train, y_test = trainy.iloc[train_], trainy.iloc[test_]
        
        model = LGBMRegressor(verbose=0,n_estimators=100,max_depth=4,
                                  random_state=27 )
        model.fit(x_train,y_train)
        preds = model.predict(x_test)
        score = mean_squared_error(y_test, preds)**(1/2)
        scores.append(score)
        realpreds = model.predict(final_test)
#         if i!=3:
        Actual_Preds.append(realpreds)
#         i+=1
        print(score)
print("AVERAGE SCORE: ", sum(scores)/len(scores))

1.4603374445580461
1.7680521771493776
1.753028585089012
1.6361562876552613
1.563063520831581
AVERAGE SCORE:  1.6361276030566558


In [36]:
preds = np.mean(Actual_Preds,0)
op2 = pd.DataFrame(preds,columns=["SalesInMillions"])

In [40]:
wholepred = op*0.5+op2*0.45   # Seemslike overestimating the model is giving good score...

In [46]:
#checking STD
display(op.SalesInMillions.std())
df_train.SalesInMillions.std()

1.2531028958082338

2.0752600525667115

In [ ]:
featimp=pd.DataFrame({"Name":model.feature_names_,
                      "Importance":model.feature_importances_})
featimp.sort_values(by=['Importance'], inplace=True, ascending=False)

In [47]:
featimp

,Name,Importance
7,AVG_POINTS,13.014118
6,USER_POINTS,12.826078
0,CONSOLE,5.147163
8,CRITICS_POINTS_max,4.366427
10,CRITICS_POINTS_count,4.018711
14,USER_POINTS_count,4.005978
5,CRITICS_POINTS,3.430607
16,CRITICS_POINTS_CATPUB_max,3.316207
12,USER_POINTS_max,2.976523
45,USER_POINTS_CATYEAR_min,2.423310


In [83]:
wholepred.to_excel("Submit_Ensemble6OOF.xlsx",index=False)